# Data collection

In this notebook we will collect the raw data from sentinel2 satellite and the target data from the CORINE data.

## Sentinel 2

The sentinel2 data is available [here](https://scihub.copernicus.eu/). This program provides free multi spectral satellite imagery of the whole planet with a refresh rate of 3 to 5 days.

The resolution depends on the band you are interested in, the highest being 10m and lowets being 60m.

In this notebook, we will use the [`sentinelsat`](https://sentinelsat.readthedocs.io/en/stable/) library to select and download our data. You will need a free [account](https://scihub.copernicus.eu/userguide/SelfRegistration) for that.

## CORINE

The CORINE program is also a European program is a manual classification of the land cover in Europe. We will be using here the 2018 update as it is the latest and as we have sentinel data for this year.

The data used is available [here](https://land.copernicus.eu/pan-european/corine-land-cover/clc2018) by going to the download tab and selecting the geoTiff. (You'll need a free account for that).

The classification contains 44 classes that you can find [here](https://land.copernicus.eu/eagle/files/eagle-related-projects/pt_clc-conversion-to-fao-lccs3_dec2010). Classes are grouped into 5 main categories:
* Artificial surfaces
* Agriculture area
* Forest and semi natural area
* Wetlands
* Water bodies

In the following notebooks, we will be interested in all kinds of forests, classes `311`, `312` ad `313`.

In the following, we assume the data is stored in the `data/` folder. We also need to have a geometry available. If you do not like the default one, you can create one on [geojson.io](http://geojson.io/). This geometry will be used to search for tiles to download.

We use here an approximate version of Normandy's Geometry as I like this region 😛

You do not need to have a super accurate geometry at this step as this geometry is only used to query input data from the sentinel satellites.

In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import zipfile
import os

# First we connect to the Copernicus public API
api = SentinelAPI(os.environ['DHUS_USER'], os.environ['DHUS_PASSWORD'])

# Then we load the geometry to query the data
footprint = geojson_to_wkt(read_geojson('../data/region.geojson'))

# We query Coppernicus for 2020 as 2018 is archived and harder to retrieve.
# We focus on June as this month has less clouds
# Here we chose data between June because it is less likely to have clouds
products = api.query(footprint,
                     date=('20200601', '20200630'),
                     producttype='S2MSI2A', # This is the athmosperic corrected version of S2
                     cloudcoverpercentage=(0, 5))
                     
print(f'{len(products)} results found')

31 results found


If you kept the default parameters you should see 31 tiles covering Normandy with less than 5% cloud coverage in June 2020. Let us now download these to the data folder. (Might take a while, so obviously try to run it only once).

We also unzip them and delete the tarball for more convenience.

In [ ]:
raw_data_path = '../data/s2a/'
api.download_all(products, raw_data_path)

for tarball in os.listdir(raw_data_path):
    path = os.path.join(raw_data_path, tarball)
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(raw_data_path)
        os.remove(path)

Downloading:   0%|          | 0.00/1.15G [00:00<?, ?B/s]
Downloading:   0%|          | 5.24M/1.15G [00:00<03:12, 5.94MB/s]
Downloading:   1%|          | 9.44M/1.15G [00:01<03:04, 6.20MB/s]
Downloading:   1%|          | 11.5M/1.15G [00:01<02:59, 6.34MB/s]
Downloading:   1%|▏         | 14.7M/1.15G [00:02<03:09, 6.01MB/s]
Downloading:   2%|▏         | 17.8M/1.15G [00:02<02:56, 6.44MB/s]
Downloading:   2%|▏         | 21.0M/1.15G [00:03<03:17, 5.72MB/s]
Downloading:   2%|▏         | 24.1M/1.15G [00:03<03:14, 5.81MB/s]
Downloading:   2%|▏         | 26.2M/1.15G [00:04<02:54, 6.44MB/s]
Downloading:   3%|▎         | 29.4M/1.15G [00:04<03:19, 5.64MB/s]
Downloading:   3%|▎         | 31.5M/1.15G [00:05<02:57, 6.32MB/s]
Downloading:   3%|▎         | 34.6M/1.15G [00:05<03:01, 6.16MB/s]
Downloading:   3%|▎         | 36.7M/1.15G [00:06<03:20, 5.57MB/s]
Downloading:   3%|▎         | 39.8M/1.15G [00:06<03:14, 5.72MB/s]
Downloading:   4%|▎         | 41.9M/1.15G [00:06<03:17, 5.61MB/s]
Downloading:   4%|▍

Downloading:  11%|█         | 124M/1.15G [00:56<03:36, 4.75MB/s]
Downloading:  12%|█▏        | 135M/1.15G [00:57<05:23, 3.14MB/s]
Downloading:  11%|█         | 126M/1.15G [00:57<03:20, 5.12MB/s]
Downloading:  12%|█▏        | 136M/1.15G [00:57<05:35, 3.03MB/s]
Downloading:  11%|█         | 128M/1.15G [00:57<03:05, 5.52MB/s]
Downloading:  11%|█         | 129M/1.15G [00:57<02:57, 5.78MB/s]
Downloading:  12%|█▏        | 137M/1.15G [00:57<05:38, 2.99MB/s]
Downloading:  11%|█▏        | 131M/1.15G [00:57<02:41, 6.31MB/s]
Downloading:  12%|█▏        | 138M/1.15G [00:58<05:57, 2.83MB/s]
Downloading:  12%|█▏        | 133M/1.15G [00:58<02:44, 6.19MB/s]
Downloading:  12%|█▏        | 134M/1.15G [00:58<02:40, 6.34MB/s]
Downloading:  12%|█▏        | 139M/1.15G [00:58<06:14, 2.70MB/s]
Downloading:  12%|█▏        | 136M/1.15G [00:58<02:53, 5.87MB/s]
Downloading:  12%|█▏        | 141M/1.15G [00:59<06:06, 2.76MB/s]
Downloading:  12%|█▏        | 138M/1.15G [00:59<02:49, 6.00MB/s]
Downloading:  12%|█▏     

For the CORINE data, please go donwload it directly on their website as indicated above and unzip it in your `data` folder. For some reason there are 2 zip layers but it's fine.

By now your data folder should look like that:

```
data
├── region.geojson
├── s2a
│   ├── S2A_MSIL2A_20200602T105631_N0214_R094_T30UWU_20200602T135525.SAFE
│   ├── S2A_MSIL2A_20200602T105631_N0214_R094_T30UXU_20200602T135525.SAFE
│   ├── S2A_MSIL2A_20200602T105631_N0214_R094_T30UXV_20200602T135525.SAFE
│   ├── S2A_MSIL2A_20200602T105631_N0214_R094_T30UYU_20200602T135525.SAFE
│   ├── S2A_MSIL2A_20200602T105631_N0214_R094_T30UYV_20200602T135525.SAFE
│   ├── S2A_MSIL2A_20200602T105631_N0214_R094_T31UCP_20200602T135525.SAFE
│   ├── S2A_MSIL2A_20200602T105631_N0214_R094_T31UCQ_20200602T135525.SAFE
│   ├── S2A_MSIL2A_20200602T105631_N0214_R094_T31UDQ_20200602T135525.SAFE
│   ├── S2A_MSIL2A_20200622T105631_N0214_R094_T30UWU_20200622T135503.SAFE
│   ├── S2A_MSIL2A_20200622T105631_N0214_R094_T30UXU_20200622T135503.SAFE
│   ├── S2A_MSIL2A_20200622T105631_N0214_R094_T30UXV_20200622T135503.SAFE
│   ├── S2A_MSIL2A_20200622T105631_N0214_R094_T30UYU_20200622T135503.SAFE
│   ├── S2A_MSIL2A_20200622T105631_N0214_R094_T30UYV_20200622T135503.SAFE
│   ├── S2A_MSIL2A_20200622T105631_N0214_R094_T31UCP_20200622T135503.SAFE
│   ├── S2A_MSIL2A_20200622T105631_N0214_R094_T31UCQ_20200622T135503.SAFE
│   ├── S2A_MSIL2A_20200625T110631_N0214_R137_T30UWA_20200625T122333.SAFE
│   ├── S2A_MSIL2A_20200625T110631_N0214_R137_T30UWV_20200625T122333.SAFE
│   ├── S2A_MSIL2A_20200625T110631_N0214_R137_T30UXA_20200625T122333.SAFE
│   ├── S2A_MSIL2A_20200625T110631_N0214_R137_T30UXU_20200625T122333.SAFE
│   ├── S2A_MSIL2A_20200625T110631_N0214_R137_T30UXV_20200625T122333.SAFE
│   ├── S2A_MSIL2A_20200625T110631_N0214_R137_T30UYA_20200625T122333.SAFE
│   ├── S2A_MSIL2A_20200625T110631_N0214_R137_T30UYV_20200625T122333.SAFE
│   ├── S2A_MSIL2A_20200625T110631_N0214_R137_T31UCQ_20200625T122333.SAFE
│   ├── S2A_MSIL2A_20200625T110631_N0214_R137_T31UCR_20200625T122333.SAFE
│   ├── S2B_MSIL2A_20200624T104629_N0214_R051_T30UYU_20200624T134207.SAFE
│   ├── S2B_MSIL2A_20200624T104629_N0214_R051_T30UYV_20200624T134207.SAFE
│   ├── S2B_MSIL2A_20200624T104629_N0214_R051_T31UCP_20200624T134207.SAFE
│   ├── S2B_MSIL2A_20200624T104629_N0214_R051_T31UCQ_20200624T134207.SAFE
│   ├── S2B_MSIL2A_20200624T104629_N0214_R051_T31UCR_20200624T134207.SAFE
│   ├── S2B_MSIL2A_20200624T104629_N0214_R051_T31UDQ_20200624T134207.SAFE
│   └── S2B_MSIL2A_20200624T104629_N0214_R051_T31UDR_20200624T134207.SAFE
└── u2018_clc2018_v2020_20u1_raster100m
    ├── DATA
    ├── Documents
    ├── French_DOMs
    ├── Legend
    └── Metadata
```

The first part of this tutorial is now over, you can go check notebook 2 on pre-processing for the following.